Primero se cargan los datos con pandas, leyendo el archivo "Vino_Tinto.csv". Se muestran las dimensiones del conjunto y se muestran las primeras cinco filas para revisar que la lectura fue correcta, verificar nombres de variables y asegurarse de que los valores tengan sentido.

In [1]:
import pandas as pd

df = pd.read_csv("Vino_Tinto.csv")
print("Dimensiones del data frame:", df.shape)
df.head()

Dimensiones del data frame: (1599, 12)


,acidezFija,acidezVolatil,acidoCitrico,azucarResidual,cloruros,dioxidoAzufreLibre,dioxidoAzufreTotal,densidad,pH,sulfatos,alcohol,calidad
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Se realiza la partición de los datos en entrenamiento y prueba. Se separa la variaable objetivo (calidad) de las variables predictoras, y se utiliza train_test_split con una division de 80% para entrenamiento y 20% para prueba. Se usa random_state=63, para que el resultado se reproducible y se impriman las dimensiones de cada subconjunto. Asi asegura que hay un conjunto de prueba independiente para evaluar la capacidad de generalizacion del modelo.

In [13]:
from sklearn.model_selection import train_test_split

target_col = "calidad" if "calidad" in df.columns else df.columns[-1]
x= df.drop(columns=[target_col])
y = df[target_col]

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=63, shuffle= True )
print("Dimensiones entrenamiento:", x_train.shape, "| prueba:", x_test.shape)
print("Total observaciones:", len(df), "| suma participaciones", len(x_train) + len(x_test))


Dimensiones entrenamiento: (1279, 11) | prueba: (320, 11)
Total observaciones: 1599 | suma participaciones 1599


Aqui se aplica el forward selection, con SequentialFeatureSelector. Para cada posible numeero de variables (k entre 2 y 8), el algoritmo contruye modelos de regresion lineal, agregando varaibles una por una de forma greedy, y mide el desempeñl usando validación cruzada de 10 folds con la metrica R². Se calcula el promedio de R² para cada subconjutno y se elige aquel que logra el valor mas alto y en caso de empate, el de menor k. Esto permite encontrar un conjunto de variables que explique bien la variacion de la calidad del vino sin incluir todas la variables disponibles.

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.feature_selection import SequentialFeatureSelector

def cv_r2_for_features(columns, x_train, y_train, cv=10):
    model = LinearRegression()
    kf = KFold(n_splits=cv, shuffle=True, random_state=63)
    return cross_val_score(model, x_train[columns], y_train, cv=kf, scoring="r2").mean()

forward_cand = range(2, min(8, x_train.shape[1]) + 1)
forward_scores = []
forward_sets = {}

for k in forward_cand:
    sfs = SequentialFeatureSelector(
        LinearRegression(),
        n_features_to_select=k,
        direction="forward",
        scoring="r2",
        cv=10
    )
    sfs.fit(x_train, y_train)
    cols = x_train.columns[sfs.get_support()].tolist()
    score_mean = cv_r2_for_features(cols, x_train, y_train, cv=10)
    forward_scores.append((k, score_mean))
    forward_sets[k] = cols

best_k, best_score = sorted(forward_scores, key=lambda t: (-t[1], t[0]))[0]
forward_features = forward_sets[best_k]

print("Mejor k (forward):", best_k)
print("Características seleccionadas (forward):", forward_features)
print("CV R^2 medio (forward):", round(best_score, 4))


Mejor k (forward): 8
Características seleccionadas (forward): ['acidezVolatil', 'acidoCitrico', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']
CV R^2 medio (forward): 0.3407


Una vez ya usado el subconjunto forward, se entrena un modelo de resion lineal multiple usando unicamente esas variables y se evalua en el conjunto de prueba. Se generan predicciones y se calcula el  R² de prueba con r2_score. Para verificar si el modelo seleccionado por validacion cruzada tambien sse desempeña bien en datos no vistos, que es lo qu realmente importa en la práctica

In [43]:
from sklearn.metrics import r2_score
fwd_model = LinearRegression().fit(x_train[forward_features], y_train)
y_pred_fwd = fwd_model.predict(x_test[forward_features])
r2_fwd = r2_score(y_test, y_pred_fwd)

print("R^2 en prueba (forward):", round(r2_fwd, 4))

R^2 en prueba (forward): 0.3678


Se usa el backward selection partiendo del conjunto de variables encontrado en forward. El algoritmo comienza con todas esas variables y elimina una por una, probando subconjuntos de tamaño k entre 2 y 5 o hasta el tamaño del conjunto forward si es menor. Se usa el 10 folds con R². Se reduce el número de variables manteniendo un buen desempeño, lo que da lugar a un modelo mas simple y facil de interpretar

In [21]:
backward_scores = []
backward_sets = {}
max_k_back = min(5, len(forward_features))
backward_cand = range(2, max_k_back + 1) if max_k_back >= 2 else [len(forward_features)]

for k in backward_cand:
    sfs_bwd = SequentialFeatureSelector(
        LinearRegression(),
        n_features_to_select=k,
        direction="backward",
        scoring="r2",
        cv=10
    )
    sfs_bwd.fit(x_train[forward_features], y_train)
    cols_bwd = pd.Index(forward_features)[sfs_bwd.get_support()].tolist()
    score_mean_bwd = cv_r2_for_features(cols_bwd, x_train, y_train, cv=10)
    backward_scores.append((k, score_mean_bwd))
    backward_sets[k] = cols_bwd

best_k_bwd, best_score_bwd = sorted(backward_scores, key=lambda t: (-t[1], t[0]))[0]
backward_features = backward_sets[best_k_bwd]

print("Mejor k (backward):", best_k_bwd)
print("Características seleccionadas (backward):", backward_features)
print("CV R^2 medio (backward):", round(best_score_bwd, 4))


Mejor k (backward): 5
Características seleccionadas (backward): ['acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol']
CV R^2 medio (backward): 0.3339


Se usa un modelo de regresión lineal con las variables seleccionadas en backward, se calcula el R² en prueba y se compara con el modelo forward. Si el backward es un R² similiar al forward pero con menos varaibles, se prefiere por simplicidad. En cambio, si el forward tiene un R² de pruea mejor, conviene quedarse con él. Utilice el if, elif y else, esto me sirve para comparar automaticamente el desempeño de los modelos y monstrar un mensaje de conclusión 

In [42]:
bwd_model = LinearRegression().fit(x_train[backward_features], y_train)
y_pred_bwd = bwd_model.predict(x_test[backward_features])
r2_bwd = r2_score (y_test, y_pred_bwd)

print("R^2 en prueba (backward):", round(r2_bwd, 4))

if r2_bwd > r2_fwd:
    print("\nConclusión: el backward tiene mejor R^2 en prueba que forward, entonces es mejor.")
elif r2_bwd < r2_fwd:
    print("\nConclusión: el forward tiene mejor R^2 en prueba que backward, entonces es mejor.")
else:
    print("\nConclusión: ambos empatan en R^2 de prueba; en tal caso favorece el de menor k.")


R^2 en prueba (backward): 0.3626

Conclusión: el forward tiene mejor R^2 en prueba que backward, entonces es mejor.


El forward obtuvo un R² en prueba ligeramente mayor en comparacion con el backward. El modelo forward tiene un desempeño mejor aunque sea poco, la diferencia en R² es minima (0.0052).
Considero que el modelo backward puede ser preferible, ya que casi logra el mismo nivel de ajuste, pero con menos variables, lo cual lo hace mas simple.